In [ ]:
!pip install opendatasets

In [1]:
import sqlalchemy
import pandas as pd
import opendatasets as od

In [5]:
engine = sqlalchemy.create_engine('sqlite:///sql_test.db')
sqlite_connection = engine.connect()
sqlite_table = 'movies'

In [3]:
def run_query(query, engine):
    return pd.read_sql(query, con=engine, index_col=None)

In [4]:
od.download('https://www.kaggle.com/datasets/anotherbadcode/boxofficecollections')

Skipping, found downloaded files in "./boxofficecollections" (use force=True to force download)


In [7]:
df = pd.read_csv('./boxofficecollections/BoxOfficeCollections.csv')

In [8]:
df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

1378

# SQL Test

#### Select the first 5 rows from the movies table

In [11]:
query = '''
select * from movies limit 5
'''
run_query(query, engine)

,Movie,Year,Score,Adjusted Score,Director,Cast,Consensus,Box Office Collection,Imdb_genre,IMDB Rating,metascore,time_minute,Votes
0,Hot Rod,2007,39,42.918,Akiva Schaffer,"Andy Samberg, Jorma Taccone, Bill Hader, Danny...","For Rod Kimball (Andy Samberg), performing stu...",14371564.0,Comedy,6.7,43.0,88.0,84956.0
1,Game Night,2018,85,99.838,John Francis Daley,"Jason Bateman, Rachel McAdams, Kyle Chandler, ...",Max and Annie's weekly game night gets kicked ...,117378084.0,Comedy,6.9,66.0,100.0,229292.0
2,The First Wives Club,1996,49,53.174,Hugh Wilson,"Goldie Hawn, Bette Midler, Diane Keaton, Maggi...",Despondent over the marriage of her ex-husband...,181489203.0,Comedy,6.4,58.0,103.0,48413.0
3,Scary Movie,2000,52,54.973,Keenen Ivory Wayans,"Shawn Wayans, Marlon Wayans, Cheri Oteri, Shan...","Defying the very notion of good taste, Scary M...",277200000.0,Comedy,6.2,48.0,88.0,254927.0
4,Blockers,2018,84,96.883,Kay Cannon,"Leslie Mann, Ike Barinholtz, John Cena, Kathry...","Julie, Kayla and Sam are three high school sen...",94523781.0,Comedy,6.2,69.0,102.0,78498.0


### Select the max rottentomatoes score

In [ ]:
query = '''

'''
run_query(query, engine)